In [ ]:
from ikrlib import png2fea
import numpy as np
from PIL import Image, ImageEnhance, ImageOps
import os
import random
import torch as t

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
def random_rotation(image):
    angle = random.randint(-30, 30)
    return image.rotate(angle)

def random_flip(image):
    flip_type = random.choice(["horizontal", "vertical"])
    if flip_type == "horizontal":
        return image.transpose(method=Image.Transpose.FLIP_LEFT_RIGHT)
    else:
        return image.transpose(method=Image.Transpose.FLIP_TOP_BOTTOM)

def random_contrast(image):
    factor = random.uniform(0.4, 1.7)
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(factor)

def augment_image(image):
    image = random_rotation(image)
    image = random_flip(image)
    image = random_contrast(image)
    return image

def random_scale(image):
    scale_factor = random.uniform(0.9, 1.1)
    width, height = image.size
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    image = image.resize((new_width, new_height), Image.LANCZOS)

    # Padding to maintain original size
    pad_x = max(0, width - new_width)
    pad_y = max(0, height - new_height)
    image = ImageOps.expand(image, (pad_x // 2, pad_y // 2, pad_x - pad_x // 2, pad_y - pad_y // 2))
    return image

def augment_images(input_dir, output_dir, num_augmentations=3):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file_name in os.listdir(input_dir):
        if file_name.endswith(".png"):
            image_path = os.path.join(input_dir, file_name)
            image = Image.open(image_path)

            for i in range(num_augmentations):
                augmented_image = augment_image(image)
                output_path = os.path.join(output_dir, f"{file_name[:-4]}_aug_{i}.png")
                augmented_image.save(output_path)
                print(f"Data augumentation, new image created: {output_path}")

augment_images("images/target_train", "images/target_train/da", 85)
augment_images("images/non_target_train", "images/non_target_train/da")

augment_images("images/target_dev", "images/target_dev/da", 85)
augment_images("images/non_target_dev", "images/non_target_dev/da")

In [ ]:
train_t = np.r_[[i.mean(axis=2) for i in png2fea("images/target_train/da").values()]]
train_n = np.r_[[i.mean(axis=2) for i in png2fea("images/non_target_train/da").values()]]
target_dev = np.r_[[i.mean(axis=2) for i in png2fea("images/target_dev/da").values()]]
non_target_dev = np.r_[[i.mean(axis=2) for i in png2fea("images/non_target_dev/da").values()]]
print("Images were successfully loaded")

In [ ]:
class BaseCNN(nn.Module):
    def __init__(self):
        super(BaseCNN, self).__init__()
        self.linear1 = nn.Linear(80 * 80, 80)
        self.linear3 = nn.Linear(80, 1)

    def forward(self, x):
        result = self.linear1(x)
        result = self.linear3(result)
        return torch.sigmoid(result)

    def prob_class_1(self, x):
        prob = self(t.from_numpy(x.astype(np.float32)))
        return prob.detach().numpy()


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 20 * 20, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 20 * 20)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x


In [ ]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 10 * 10, 256)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 10 * 10)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [ ]:
# Convert NumPy arrays to PyTorch tensors
train_t_tensors = torch.Tensor(train_t).unsqueeze(1)
train_n_tensors = torch.Tensor(train_n).unsqueeze(1)
target_dev_tensors = torch.Tensor(target_dev).unsqueeze(1)
non_target_dev_tensors = torch.Tensor(non_target_dev).unsqueeze(1)

# Modify the labels' shape
train_labels = torch.Tensor(np.concatenate((np.ones(len(train_t)), np.zeros(len(train_n))))).unsqueeze(1)
dev_labels = torch.Tensor(np.concatenate((np.ones(len(target_dev)), np.zeros(len(non_target_dev))))).unsqueeze(1)

# Create new TensorDataset instances with the modified labels
train_dataset = TensorDataset(torch.cat((train_t_tensors, train_n_tensors)), train_labels)
dev_dataset = TensorDataset(torch.cat((target_dev_tensors, non_target_dev_tensors)), dev_labels)


batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Instantiate model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=5e-6)

# Training loop
num_epochs = 40
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

    train_loss = train_loss / len(train_loader.dataset)

    # Evaluation on the dev set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dev_loader:
            outputs = model(inputs)
            print(outputs)
            predicted = (outputs > 0.50).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch: {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}')

In [26]:
eval_t = np.r_[[i.mean(axis=2) for i in png2fea("images/target_dev/da").values()]]
eval_n = np.r_[[i.mean(axis=2) for i in png2fea("images/non_target_dev/da").values()]]

eval_t_tensors = torch.Tensor(eval_t).unsqueeze(1)
eval_n_tensors = torch.Tensor(eval_n).unsqueeze(1)

eval_labels = torch.Tensor(np.concatenate((np.ones(len(eval_t)), np.zeros(len(eval_n))))).unsqueeze(1)
eval_dataset = TensorDataset(torch.cat((eval_t_tensors, eval_n_tensors)), eval_labels)

batch_size = 1
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)

model.eval()
with torch.no_grad():
    for inputs, labels in eval_loader:
        outputs = model(inputs)
        print(outputs)
        predicted = (outputs > 0.50).float()
        print(labels)
        print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy of the model on the test dataset: {accuracy * 100:.2f}%')

Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_3.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_60.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_74.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_48.png
Processing file:  images/target_dev/da/m432_04_f18_i0_0_aug_27.png
Processing file:  images/target_dev/da/m432_04_f18_i0_0_aug_33.png
Processing file:  images/target_dev/da/m432_04_f18_i0_0_aug_32.png
Processing file:  images/target_dev/da/m432_04_f18_i0_0_aug_26.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_49.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_75.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_61.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_2.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_0.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_77.png
Processing file:  images/target_dev/da/m432_04_r09_i0_0_aug_63.pn